In [1]:
%load_ext autoreload
%autoreload 2
import numpy as np
from pydantic.dataclasses import dataclass, Field
import dotenv
import interlab
from interlab.queries import query_for_json, query_model
import langchain_community.chat_models
from treetrace import TracingNode, FileStorage
from interlab_zoo.persona.personas import Factory
from interlab_zoo.persona.contacts import link
from interlab_zoo.persona.channels import ChatChannel
from interlab_zoo.resolve.resolve import Situation, and_join
from serde.yaml import from_yaml
import pandas as pd
import langchain
import xml.etree.ElementTree as ET
import uuid
import itertools

dotenv.load_dotenv()

True

In [3]:
#from langchain.cache import InMemoryCache
#from langchain.globals import set_llm_cache
#set_llm_cache(InMemoryCache())


In [2]:

def resolve_direct(sitation, channel=ChatChannel, max_steps_per_player=7):
    players = sitation.players
    link(players)
    for i in range(max_steps_per_player):
        for j, sender in enumerate(players):
            others = players[:j] + players[j+1:]
            channel.query_and_send_message_to_all(sender, others, [sitation.judge])
        if sitation.agreement_check():
            return


def resolve_with_mediator(sitation, mediator_key, channel=ChatChannel, max_steps_per_player=7):
    names = and_join(p.name for p in sitation.players)
    mediator = sitation.factory.create_persona(
        mediator_key,
        public=f"$name is a mediator to arrange arrange an agreement between {names}",
        goal=f"to help {names} find an agreement on {sitation.subject}; you will not watch the movie")
    players = sitation.players
    link(players + [mediator])
    for i in range(max_steps_per_player):
        for j, sender in enumerate(players):
            others = players[:j] + players[j+1:]
            channel.query_and_send_message(sender, mediator, others + [sitation.judge])
        channel.query_and_send_message_to_all(mediator, players, [sitation.judge])
        if sitation.agreement_check():
            return


# with TracingNode("root") as root:
#     resolver = Sitation("a specific movie name they want to watch together", factory, ["Maria-gpt35", "Connor-gpt35"])

#     with TracingNode("resolving"):
#         resolve_direct(resolver, "Edward-gpt35")
#     print(resolver.agreement_result)
#     with TracingNode("scoring"):
#         scoring = resolver.score_result()
#     df = pd.DataFrame(scoring, columns=["key", "result", "score_basic", "score_wh"])
#     print(df)
    
#root.display()

In [3]:
storage = FileStorage("logs")
storage.start_server()

<ServerHandle http://localhost:46403>

In [4]:
@dataclass
class Experiment:
    name: str
    people: str
    subject: str
    repeats: int
    permutations: bool
    groups: list[list[str]]

def experiments_from_yaml(path):
    with open(path) as f:
        data = f.read()        
    experiments = from_yaml(list[Experiment], data)
    return {e.name: e for e in experiments}

experiments = experiments_from_yaml("experiments.yaml")
experiments

{'movies': Experiment(name='movies', people='people-movies.yaml', subject='a single specific movie name they want to watch together', repeats=2, permutations=True, groups=[['Maria-gpt35', 'Jane-gpt35'], ['Don-gpt35', 'Jane-gpt35'], ['Don-gpt35', 'Jane-gpt35'], ['Edward-gpt35', 'Connor-gpt35'], ['Edward-gpt35', 'Jane-gpt35'], ['Lilly-gpt35', 'Edward-gpt35']]),
 'board_games': Experiment(name='board_games', people='people-bg.yaml', subject='a single specific board game to play together', repeats=5, permutations=True, groups=[['Carl-gpt35', 'Emma-gpt35'], ['John-gpt35', 'Sofia-gpt35'], ['Carl-gpt35', 'Sofia-gpt35'], ['Emma-gpt35', 'Sofia-gpt35'], ['Carl-gpt35', 'John-gpt35']])}

In [ ]:
def resolve_situation(situation, method):
    with TracingNode("resolving") as node:
        method(situation)
        node.set_result(situation.agreement_result)
    if situation.agreement_result is None:
        return None
    with TracingNode("scoring"):
        scoring = situation.score_result()
        return pd.DataFrame(scoring, columns=["person_key", "result", "score_basic", "score_wh"])    

def run_experiment(exp):
    result = []
    with TracingNode(f"Experiment {exp.name}") as root:
        factory = Factory.from_yaml(exp.people)
        for group in exp.groups:
            if exp.permutations:
                g_iter = itertools.permutations(group)
            else:
                g_iter = iter([group])
            for people in g_iter:
                for method_name, method_fn in [("mediator", lambda s: resolve_with_mediator(s, "Mediator")),
                                               ("direct", resolve_direct),
                                              ]:
                    for _ in range(exp.repeats):
                        uid = uuid.uuid4().hex
                        with TracingNode(f"experiment instance {uid}", inputs={
                            "uid": uid,
                            "method": method_name,
                            "experiment": exp.name,
                            "people": people
                        }):
                            situation = Situation(exp.subject, factory, people)    
                            df = resolve_situation(situation, method_fn)
                            if df is not None:
                                df["method"] = method_name
                                df["experiment"] = exp.name
                                df["instance"] = uid
                                df["group"] = ",".join(people)
                                result.append(df)
    return pd.concat(result)

exp_name = "board_games"
with TracingNode("root", storage=storage) as root:
    df = run_experiment(experiments[exp_name])
    df.to_csv(f"out-{exp_name}-1.csv", index=False)
df

/home/ada/.cache/pypoetry/virtualenvs/interlab-cZHazbI_-py3.10/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.openai.ChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(
/home/ada/.cache/pypoetry/virtualenvs/interlab-cZHazbI_-py3.10/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.7 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(
